In [1]:
import pymongo

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [3]:
db = client["Global_Coffee_Consumption"]

In [4]:
collection_names = db.list_collection_names()

In [5]:
print(collection_names)

['Coffee_exports', 'Coffee_DomesticComsumption', 'Coffee_production', 'Coffee_ImportersConsumption', 'Coffee_imports ']
